In [28]:
from osgeo import gdal 
from osgeo import ogr 

# Parameter filepaths
dem_fp = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\rovaniemi.tif"
vec_fp = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\T4324R.shp\r_T4324R_p.shp"
out_fp= r'C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\output_dem'

# Open both raster and vector layers with gdal and ogr
ras_ds = gdal.Open(dem_fp) 
vec_ds = ogr.Open(vec_fp) 
lyr = vec_ds.GetLayer()

# Get geotransformation information
geot = ras_ds.GetGeoTransform()

# Create empty geotiff with same extent and resolution as the input dem
driver = gdal.GetDriverByName("GTiff") 
chn_ras_ds = driver.Create(out_fp, ras_ds.RasterXSize, ras_ds.RasterYSize, 1, gdal.GDT_Float32)
chn_ras_ds.SetGeoTransform(geot)

# Rasterize given vector file with same resolution as input dem 
gdal.RasterizeLayer(chn_ras_ds, [1], lyr,)#options=['ATTRIBUTE=Height']) 
chn_ras_ds.GetRasterBand(1).SetNoDataValue(0.0) 
chn_ras_ds= None

In [53]:
import laspy
import numpy as np 
import rasterio
from rasterio.windows import Window
from rasterio.features import geometry_mask
from shapely.geometry import Polygon

# Classify witch points class are used from point cloud
classification = 2
raster_buildings = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\output_dem_kopio"
laz_file = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\T4324E3.laz"
# Read LAZ file using laspy and raster file with rasterio
with laspy.open(laz_file) as laz, rasterio.open(raster_buildings) as src:

    # Decompress laz to las file
    las_file = laz.read()
    # Test if the count of uncompressed data is the same as compressed
    # -> NO data lost on decompression
    if  laz.header.point_count != len(las_file.points):
        raise ValueError (f'Points from laz header count({laz.header.point_count}) and points from data count({len(las_file.points)}) do not match')

    class_mask = las_file.classification == classification
    filtered_points = laspy.create_las(data=las_file.points[class_mask])

    # Get the raster profile and transformation
    profile = src.profile
    transform = src.transform

    # Loop through each pixel in the raster
    for row in range(src.height):
        for col in range(src.width):
            # Check if the pixel value is 255
            
            pixel_value = src.read(1, window=Window.from_slices((row,row+1),(col,col+1)))[0][0]
            y = Window.from_slices((row,row+1),(col,col+1))
            x = src.window(col, row, col + 1, row + 1)
            if pixel_value != 255:
                continue
            pixel_coords = [(col, row), (col + 1, row), (col + 1, row + 1), (col, row + 1)]
    
             # Transform pixel coordinates to raster coordinates
            raster_coords = [src.xy(pixel_coord[1], pixel_coord[0]) for pixel_coord in pixel_coords]
    
            # Create polygon from raster coordinates
            polygon = Polygon(pixel_coords)
            # Get the pixel geometry and mask the LAS points with it
            pixel_geom = geometry_mask([polygon], out_shape=src.shape, transform=transform)
            print(type(filtered_points))
            las_points_mask = filtered_points.get_query_geometry(polygon)
            break
            # If there are no LAS points in the pixel, skip to the next one
            if len(las_points_mask) == 0:
                continue

            # Calculate the average height of LAS points within the pixel
            las_points = filtered_points.points[las_points_mask]
            average_height = las_points['Z'].mean()

            # Write the average height to the raster pixel
            src.write(average_height, 1, window=src.window(col, row, col + 1, row + 1))
        break
# Close the LAS file
laz.close()
src.close()
        



AttributeError: module 'laspy' has no attribute 'create_las'

In [34]:
help(src)

Help on DatasetReader in module rasterio.io object:

class DatasetReader(rasterio._io.DatasetReaderBase, rasterio.windows.WindowMethodsMixin, rasterio.transform.TransformMethodsMixin)
 |  An unbuffered data and metadata reader
 |  
 |  Method resolution order:
 |      DatasetReader
 |      rasterio._io.DatasetReaderBase
 |      rasterio._base.DatasetBase
 |      rasterio.windows.WindowMethodsMixin
 |      rasterio.transform.TransformMethodsMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from rasterio._io.DatasetReaderBase:
 |  
 |  __reduce__ = __reduce_cython

In [33]:
src.close()